In [ ]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

!pip install ultralytics
!pip install roboflow
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122

# Import required libraries
from ultralytics import YOLO
import matplotlib.pyplot as plt
from roboflow import Roboflow
import os
import yaml
import json

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-

In [ ]:
# Initialize Roboflow with API key
rf = Roboflow(api_key="HA7uKn41BYAjKQOtGKJz")
project = rf.workspace("capstone-ncwsi").project("test-5dvot")
version = project.version("17")

# download dataset
dataset = version.download("yolov8")

print("Dataset location:", dataset.location)
print("Contents of dataset location:")
print(os.listdir(dataset.location))

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.68, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to test-17 in yolov8:: 100%|██████████| 14386/14386 [00:02<00:00, 5239.28it/s]

Dataset location: /content/test-17
Contents of dataset location:
['test', 'README.roboflow.txt', 'valid', 'data.yaml', 'README.dataset.txt', 'train']


Construct the path to the YAML file that contains dataset configuration.

In [ ]:
yaml_path = os.path.join(dataset.location, "data.yaml")

with open(yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)
print(data_config)

print("YAML Path:", yaml_path)

with open(yaml_path, 'r') as f:
    print(f.read())


{'names': ['Beetle-fire Tree', 'Dead Tree', 'debris'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'test-5dvot', 'url': 'https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/17', 'version': 17, 'workspace': 'capstone-ncwsi'}, 'test': '../test/images', 'train': 'test-17/train/images', 'val': 'test-17/valid/images'}
YAML Path: /content/test-17/data.yaml
names:
- Beetle-fire Tree
- Dead Tree
- debris
nc: 3
roboflow:
  license: CC BY 4.0
  project: test-5dvot
  url: https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/17
  version: 17
  workspace: capstone-ncwsi
test: ../test/images
train: test-17/train/images
val: test-17/valid/images



Update the dictionary with new paths for the 'train', 'val', and 'test' keys.

In [ ]:
import yaml

yaml_path = '/content/test-17/data.yaml'
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

data['train'] = 'train/images'
data['val'] = 'valid/images'
data['test'] = 'test/images'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print("YAML file updated.")

with open(yaml_path, 'r') as f:
    print(yaml.safe_load(f))

YAML file updated.
{'names': ['Beetle-fire Tree', 'Dead Tree', 'debris'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'test-5dvot', 'url': 'https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/17', 'version': 17, 'workspace': 'capstone-ncwsi'}, 'test': 'test/images', 'train': 'train/images', 'val': 'valid/images'}


List and print the contents.

In [ ]:
import yaml

with open('/content/test-17/data.yaml', 'r') as f:
    data = yaml.safe_load(f)
    print(data)

import os

print(os.listdir('/content/test-17'))
print(os.listdir('/content/test-17/train'))

{'names': ['Beetle-fire Tree', 'Dead Tree', 'debris'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'test-5dvot', 'url': 'https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/17', 'version': 17, 'workspace': 'capstone-ncwsi'}, 'test': 'test/images', 'train': 'train/images', 'val': 'valid/images'}
['test', 'README.roboflow.txt', 'valid', 'data.yaml', 'README.dataset.txt', 'train']
['images', 'labels']


In [ ]:
import os

def print_directory_structure(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files[:5]:
            print('{}{}'.format(subindent, f))
        if len(files) > 5:
            print('{}...'.format(subindent))

print_directory_structure('/content/test-17')

test-17/
    data_small.yaml
    README.roboflow.txt
    data.yaml
    README.dataset.txt
    test/
        images/
            DJI_0039_JPG_jpg.rf.f21afc4272b7c511a8b9ba39bd7cb83a.jpg
            DJI_0179_JPG_jpg.rf.42e2e63f6bc5aa9128b8b31376576136.jpg
            DJI_0087_JPG_jpg.rf.368cba42a5bb2e26f2adde2e399d8ac1.jpg
            DJI_0274_JPG_jpg.rf.aad9bf6795e9f58b4ce6658edda0413c.jpg
            DJI_0027_JPG_jpg.rf.eabe2fee7be57c26e606e2b6da268982.jpg
            ...
        labels/
            DJI_0064_JPG_jpg.rf.710b0ae3d04ebc0ed1781e9d24f214f9.txt
            DJI_0078_JPG_jpg.rf.89f4c2a513f794dc96289c2fcad30cd7.txt
            DJI_0414_JPG_jpg.rf.2182f14a081b9adda3febafaad2a9b73.txt
            DJI_0536_JPG_jpg.rf.a2487e502fd345281f737339c4941ca7.txt
            DJI_0876_JPG_jpg.rf.2876ff807979467849be62788104c610.txt
            ...
    valid/
        labels.cache
        images/
            DJI_0362_JPG_jpg.rf.18099e93f0a755381b6744c08e3d3ec2.jpg
            DJI_0003_JPG_jpg.

We creates a smaller version of an existing dataset.

In [ ]:
import random
from pathlib import Path
import yaml
import shutil
import os

def create_small_dataset(original_yaml_path, small_dataset_ratio=0.2):
    with open(original_yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    base_dir = Path(original_yaml_path).parent
    original_train = base_dir / data['train']
    original_val = base_dir / data['val']

    small_train = original_train.parent / (original_train.name + '_small')
    small_val = original_val.parent / (original_val.name + '_small')

    for orig, small in [(original_train, small_train), (original_val, small_val)]:
        print(f"Processing {orig} to {small}")
        if orig.is_dir():
            os.makedirs(small, exist_ok=True)
            image_files = list(orig.glob('*.jpg')) + list(orig.glob('*.png'))
            print(f"Found {len(image_files)} images in {orig}")
            sampled_files = random.sample(image_files, max(1, int(len(image_files) * small_dataset_ratio)))
            print(f"Sampled {len(sampled_files)} images")

            for img_file in sampled_files:
                dest_img = small / img_file.name
                print(f"Copying {img_file} to {dest_img}")
                try:
                    shutil.copy(img_file, dest_img)
                except shutil.SameFileError:
                    print(f"Skipping {img_file} as it's the same as the destination")
                except Exception as e:
                    print(f"Error copying {img_file}: {e}")

                # copy label file
                label_file = orig.parent / 'labels' / (img_file.stem + '.txt')
                if label_file.exists():
                    os.makedirs(small.parent / 'labels', exist_ok=True)
                    dest_label = small.parent / 'labels' / (img_file.stem + '.txt')
                    print(f"Copying {label_file} to {dest_label}")
                    try:
                        shutil.copy(label_file, dest_label)
                    except shutil.SameFileError:
                        print(f"Skipping {label_file} as it's the same as the destination")
                    except Exception as e:
                        print(f"Error copying {label_file}: {e}")
                else:
                    print(f"Label file not found: {label_file}")
        else:
            print(f"Warning: {orig} is not a directory. Skipping.")

    small_yaml_path = base_dir / (Path(original_yaml_path).stem + '_small.yaml')
    data['train'] = str(small_train.relative_to(base_dir))
    data['val'] = str(small_val.relative_to(base_dir))

    with open(small_yaml_path, 'w') as f:
        yaml.dump(data, f)

    return str(small_yaml_path)


original_yaml_path = '/content/test-17/data.yaml'
small_yaml_path = create_small_dataset(original_yaml_path)
print(f"Small dataset created: {small_yaml_path}")

# verify small dataset
with open(small_yaml_path, 'r') as f:
    small_data = yaml.safe_load(f)
    print("Small dataset YAML content:", small_data)

base_dir = Path(original_yaml_path).parent
print("Small train directory content:", os.listdir(base_dir / small_data['train']))
print("Small valid directory content:", os.listdir(base_dir / small_data['val']))

Processing /content/test-17/train/images to /content/test-17/train/images_small
Found 6282 images in /content/test-17/train/images
Sampled 1256 images
Copying /content/test-17/train/images/DJI_0421_JPG_jpg.rf.29b8476de47669c749334d23071b88e1.jpg to /content/test-17/train/images_small/DJI_0421_JPG_jpg.rf.29b8476de47669c749334d23071b88e1.jpg
Copying /content/test-17/train/labels/DJI_0421_JPG_jpg.rf.29b8476de47669c749334d23071b88e1.txt to /content/test-17/train/labels/DJI_0421_JPG_jpg.rf.29b8476de47669c749334d23071b88e1.txt
Skipping /content/test-17/train/labels/DJI_0421_JPG_jpg.rf.29b8476de47669c749334d23071b88e1.txt as it's the same as the destination
Copying /content/test-17/train/images/DJI_0078_JPG_jpg.rf.f8dfcd2b392714ead2c71e5c9988c08d.jpg to /content/test-17/train/images_small/DJI_0078_JPG_jpg.rf.f8dfcd2b392714ead2c71e5c9988c08d.jpg
Copying /content/test-17/train/labels/DJI_0078_JPG_jpg.rf.f8dfcd2b392714ead2c71e5c9988c08d.txt to /content/test-17/train/labels/DJI_0078_JPG_jpg.rf.f8

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
from pathlib import Path
import yaml
import logging
import os

# set log
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

class CustomYOLO(YOLO):
    def __init__(self, model='yolov8m.pt', task=None):
        super().__init__(model, task)
        self.focal_loss = FocalLoss(alpha=0.25, gamma=2.0)

    def _load(self, weights='yolov8m.pt', task='detect'):
        super()._load(weights, task)
        # modify loss
        if hasattr(self.model, 'model') and hasattr(self.model.model, 'loss'):
            original_loss_func = self.model.model.loss
            def custom_loss(*args, **kwargs):
                loss, loss_items = original_loss_func(*args, **kwargs)
                # replace to focal loss
                pred_scores, target_scores = args[0][1], args[1]['cls']
                focal_loss = self.focal_loss(pred_scores, target_scores)
                loss_items[1] = focal_loss
                loss = loss_items.sum()
                return loss, loss_items
            self.model.model.loss = custom_loss

def grid_search_focal_loss(yaml_path, save_dir, alphas, gammas, epochs=10):
    results = {}
    for alpha in alphas:
        for gamma in gammas:
            logging.info(f"Testing alpha={alpha}, gamma={gamma}")
            try:
                model = CustomYOLO('yolov8m.pt')
                model.focal_loss = FocalLoss(alpha=alpha, gamma=gamma)

                train_results = model.train(
                    data=yaml_path,
                    epochs=epochs,
                    imgsz=640,
                    batch=16,
                    lr0=0.001,
                    lrf=0.01,
                    momentum=0.937,
                    weight_decay=0.001,
                    warmup_epochs=5,
                    warmup_bias_lr=0.1,
                    warmup_momentum=0.8,
                    box=10.0,
                    cls=1.0,
                    dfl=1.5,
                    patience=100,
                    optimizer="AdamW",
                    cos_lr=True,
                    mixup=0.5,
                    copy_paste=0.3,
                    degrees=15.0,
                    hsv_h=0.015,
                    hsv_s=0.7,
                    hsv_v=0.4,
                    translate=0.2,
                    scale=0.6,
                    shear=3.0,
                    perspective=0.0005,
                    flipud=0.5,
                    fliplr=0.5,
                    mosaic=1.0,
                    conf=0.001,
                    iou=0.5,
                    verbose=True,
                    save_period=10,
                    label_smoothing=0.1,
                    project=str(save_dir),
                    name=f'focal_loss_a{alpha}_g{gamma}',
                    exist_ok=True
                )

                # evaluate model
                val_results = model.val(data=yaml_path)

                results[(alpha, gamma)] = {
                    'mAP50': val_results.box.map50,
                    'mAP50-95': val_results.box.map,
                    'precision': val_results.box.p,
                    'recall': val_results.box.r
                }

                logging.info(f"Finished alpha={alpha}, gamma={gamma}")

                # save results
                with open(save_dir / 'intermediate_results.yaml', 'w') as f:
                    yaml.dump(results, f)

            except Exception as e:
                logging.error(f"Error occurred for alpha={alpha}, gamma={gamma}: {str(e)}")

    return results

if __name__ == "__main__":
    yaml_path = '/content/test-17/data.yaml'
    save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search')


    os.makedirs(save_dir, exist_ok=True)

    alphas = [0.25, 0.5, 0.75]
    gammas = [1.0, 2.0, 3.0]

    search_results = grid_search_focal_loss(yaml_path, save_dir, alphas, gammas)

    # save the final results
    with open(save_dir / 'final_results.yaml', 'w') as f:
        yaml.dump(search_results, f)

    logging.info("Grid search completed. Results saved.")

Ultralytics YOLOv8.2.68 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/test-17/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=/content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search, name=focal_loss_a0.25_g1.0, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.001, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

100%|██████████| 755k/755k [00:00<00:00, 27.4MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 119MB/s]


AMP: checks passed ✅


...


10 epochs completed in 0.187 hours.

10 epochs completed in 0.187 hours.
Optimizer stripped from /content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0/weights/best.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0/weights/best.pt...

Validating /content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0/weights/best.pt...
Ultralytics YOLOv8.2.68 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Ultralytics YO

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  4.13it/s]



                   all        607       1447       0.42      0.365      0.321      0.115
                   all        607       1447       0.42      0.365      0.321      0.115
      Beetle-fire Tree         63         79      0.485      0.441      0.384      0.145
      Beetle-fire Tree         63         79      0.485      0.441      0.384      0.145
             Dead Tree        297        550      0.501      0.435      0.402      0.148
             Dead Tree        297        550      0.501      0.435      0.402      0.148
                debris        265        818      0.274      0.221      0.177     0.0534
                debris        265        818      0.274      0.221      0.177     0.0534
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 3.6ms postprocess per image
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0
Results saved 

val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:05<00:00,  7.19it/s]



                   all        607       1447      0.419      0.364      0.321      0.115
                   all        607       1447      0.419      0.364      0.321      0.115
      Beetle-fire Tree         63         79      0.483      0.438      0.385      0.145
      Beetle-fire Tree         63         79      0.483      0.438      0.385      0.145
             Dead Tree        297        550        0.5      0.434      0.403      0.148
             Dead Tree        297        550        0.5      0.434      0.403      0.148
                debris        265        818      0.273       0.22      0.177     0.0531
                debris        265        818      0.273       0.22      0.177     0.0531
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 3.3ms postprocess per image
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0
Results saved 

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install ruamel.yaml
import matplotlib.pyplot as plt
import numpy as np
import yaml
from ruamel.yaml import YAML
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')


def load_results(file_path):
    yaml = YAML(typ='unsafe')
    with open(file_path, 'r') as f:
        return yaml.load(f)

def analyze_results(results):
    print("Starting analysis...")
    metrics = ['mAP50', 'mAP50-95', 'precision', 'recall']
    alphas = sorted(set(a for a, _ in results.keys()))
    gammas = sorted(set(g for _, g in results.keys()))

    print("Metrics:", metrics)
    print("Alphas:", alphas)
    print("Gammas:", gammas)

    for metric in metrics:
        plt.figure(figsize=(12, 8))

        for alpha in alphas:
            x = [g for a, g in results.keys() if a == alpha]
            y = [results[(a, g)][metric] for a, g in results.keys() if a == alpha]
            if isinstance(y[0], (np.ndarray, list)):
                y = [np.mean(v) if isinstance(v, np.ndarray) else sum(v)/len(v) for v in y]
            plt.plot(x, y, marker='o', label=f'alpha={alpha}')

        plt.title(f'{metric} vs Gamma for different Alpha values')
        plt.xlabel('Gamma')
        plt.ylabel(metric)
        plt.legend()
        plt.grid(True)
        plt.savefig(f'{metric}_vs_gamma.png')
        plt.close()


    for metric in metrics:
        plt.figure(figsize=(10, 8))
        heatmap_data = np.zeros((len(alphas), len(gammas)))

        for i, alpha in enumerate(alphas):
            for j, gamma in enumerate(gammas):
                value = results[(alpha, gamma)][metric]
                if isinstance(value, (np.ndarray, list)):
                    value = np.mean(value) if isinstance(value, np.ndarray) else sum(value)/len(value)
                heatmap_data[i, j] = value

        plt.imshow(heatmap_data, cmap='hot', interpolation='nearest')
        plt.colorbar()
        plt.title(f'{metric} Heatmap')
        plt.xlabel('Gamma')
        plt.ylabel('Alpha')
        plt.xticks(range(len(gammas)), gammas)
        plt.yticks(range(len(alphas)), alphas)

        for i in range(len(alphas)):
            for j in range(len(gammas)):
                plt.text(j, i, f'{heatmap_data[i, j]:.2f}',
                         ha="center", va="center", color="w")

        plt.savefig(f'{metric}_heatmap.png')
        plt.close()

    # find the best parameters
    best_params = {}
    for metric in metrics:
        best_combo = max(results.items(), key=lambda x: np.mean(x[1][metric]) if isinstance(x[1][metric], (np.ndarray, list)) else x[1][metric])
        best_value = np.mean(best_combo[1][metric]) if isinstance(best_combo[1][metric], (np.ndarray, list)) else best_combo[1][metric]
        best_params[metric] = {'alpha': best_combo[0][0], 'gamma': best_combo[0][1], 'value': best_value}

    print("Best parameter combinations for each metric:")
    for metric, params in best_params.items():
        print(f"{metric}: Alpha = {params['alpha']}, Gamma = {params['gamma']}, Value = {params['value']:.4f}")


save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_focal_loss_checkpoints/grid_search')
results_file = save_dir / 'final_results.yaml'

print("Checking if results file exists:", results_file.exists())

if results_file.exists():
    print("Results file found. Loading and analyzing results:")
    search_results = load_results(results_file)
    analyze_results(search_results)
else:
    print(f"Results file not found: {results_file}")
    print("Please check the file path and ensure the grid search results were saved.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 29.9 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checking if results file exists: True
Results file found. Loading and analyzing results:
Starting analysis...
Metrics: ['mAP50', 'mAP50-95', 'precision', 'recall']
Alphas: [0.25, 0.5, 0.75]
Gammas: [1.0, 2.0, 3.0]
Best parameter combinations for each metric:
mAP50: Alpha = 0.25, Gamma = 1.0, Value = 0.3214
mAP50-95: Alpha = 0.25, Gamma = 1.0, Value = 0.1154
precision: Alpha = 0.25, Gamma = 1.0, Value = 0.4190
recall: Alpha = 0.5, Gamma = 1.0, Value = 0.3693


In [ ]:
from google.colab import drive
drive.flush_and_unmount()

!rm -rf /content/drive


Drive not mounted, so nothing to flush and unmount.


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pip install ultralytics
!pip install roboflow
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122

import os
import yaml
import json
import shutil
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
import matplotlib.pyplot as plt
from roboflow import Roboflow

# Initialize Roboflow with API key
rf = Roboflow(api_key="HA7uKn41BYAjKQOtGKJz")
project = rf.workspace("capstone-ncwsi").project("test-5dvot")
version = project.version("17")

dataset = version.download("yolov8")

# Now we can use dataset.location
yaml_path = os.path.join(dataset.location, "data.yaml")

def print_directory_structure(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files[:5]:  # Limiting to first 5 files to avoid clutter
            print('{}{}'.format(subindent, f))
        if len(files) > 5:
            print('{}...'.format(subindent))

print("Directory structure:")
print_directory_structure(dataset.location)

def fix_yaml_paths(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    base_dir = os.path.dirname(yaml_path)
    data['train'] = os.path.relpath(os.path.join(base_dir, 'train', 'images'), base_dir)
    data['val'] = os.path.relpath(os.path.join(base_dir, 'valid', 'images'), base_dir)
    data['test'] = os.path.relpath(os.path.join(base_dir, 'test', 'images'), base_dir)

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)

    print("YAML file updated.")

fix_yaml_paths(yaml_path)

print("\nYAML file content:")
with open(yaml_path, 'r') as f:
    print(f.read())

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

class CustomYOLO(YOLO):
    def __init__(self, model='yolov8m.pt', task=None):
        super().__init__(model, task)
        self.focal_loss = FocalLoss(alpha=0.25, gamma=2.0)

    def _load(self, weights='yolov8m.pt', task='detect'):
        super()._load(weights, task)

        if hasattr(self.model, 'model') and hasattr(self.model.model, 'loss'):
            original_loss_func = self.model.model.loss
            def custom_loss(*args, **kwargs):
                loss, loss_items = original_loss_func(*args, **kwargs)

                pred_scores, target_scores = args[0][1], args[1]['cls']
                focal_loss = self.focal_loss(pred_scores, target_scores)
                loss_items[1] = focal_loss
                loss = loss_items.sum()
                return loss, loss_items
            self.model.model.loss = custom_loss

def on_train_epoch_end(trainer):
    epoch = trainer.epoch
    if epoch % trainer.args.save_period == 0:
        best_fitness = trainer.best_fitness if trainer.best_fitness is not None else 0.0
        extra_info = {
            'epoch': epoch,
            'best_fitness': float(best_fitness),
            'learning_rate': float(trainer.optimizer.param_groups[0]['lr']),
        }
        save_dir = Path(trainer.args.project) / trainer.args.name
        info_path = save_dir / f'extra_info_epoch_{epoch}.json'
        latest_info_path = save_dir / 'extra_info_latest.json'
        with open(info_path, 'w') as f:
            json.dump(extra_info, f)
        with open(latest_info_path, 'w') as f:
            json.dump(extra_info, f)

        # Save a backup of the checkpoint
        checkpoint_path = save_dir / 'weights' / f'epoch{epoch}.pt'
        backup_path = save_dir / 'weights' / f'epoch{epoch}_backup.pt'
        if checkpoint_path.exists():
            shutil.copy(str(checkpoint_path), str(backup_path))

def train_and_evaluate_with_focal_loss(yaml_path, save_dir, alpha, gamma):
    total_epochs = 200
    weights_dir = save_dir / 'focal_loss_final' / 'weights'

    # Start a new training session
    print("Starting a new training session.")
    model = CustomYOLO('yolov8m.pt')
    model.focal_loss = FocalLoss(alpha=alpha, gamma=gamma)


    os.makedirs(str(weights_dir), exist_ok=True)

    model.add_callback('on_train_epoch_end', on_train_epoch_end)

    results = model.train(
        data=yaml_path,
        epochs=total_epochs,
        imgsz=640,
        batch=16,
        lr0=0.001,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.001,
        warmup_epochs=5,
        warmup_bias_lr=0.1,
        warmup_momentum=0.8,
        box=10.0,
        cls=1.0,
        dfl=1.5,
        patience=100,
        optimizer="AdamW",
        cos_lr=True,
        mixup=0.5,
        copy_paste=0.3,
        degrees=15.0,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        translate=0.2,
        scale=0.6,
        shear=3.0,
        perspective=0.0005,
        flipud=0.5,
        fliplr=0.5,
        mosaic=1.0,
        conf=0.001,
        iou=0.5,
        verbose=True,
        save_period=10,
        label_smoothing=0.1,
        project=str(save_dir),
        name='focal_loss_final',
        exist_ok=True,
        resume=False,
        device='0'  # Specify the GPU device if needed
    )

    # Evaluate model
    val_results = model.val(data=yaml_path)

    return results, val_results

# Set the save directory
save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329')

# Set the best alpha and gamma values
best_alpha = 0.25
best_gamma = 1.0

# Run the training and evaluation
results, val_results = train_and_evaluate_with_focal_loss(yaml_path, save_dir, best_alpha, best_gamma)

# Print final validation results
if val_results:
    print(f"\nFinal validation results:")
    print(f"mAP50: {val_results.box.map50:.4f}")
    print(f"mAP50-95: {val_results.box.map:.4f}")
    print(f"Precision: {val_results.box.p:.4f}")
    print(f"Recall: {val_results.box.r:.4f}")
else:
    print("Validation results not available.")

Mounted at /content/drive
Looking in indexes: https://download.pytorch.org/whl/cu122
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.69, to fix: `pip install ultralytics==8.0.196`
Directory structure:
test-17/
    README.roboflow.txt
    data.yaml
    README.dataset.txt
    test/
        images/
            DJI_0039_JPG_jpg.rf.f21afc4272b7c511a8b9ba39bd7cb83a.jpg
            DJI_0179_JPG_jpg.rf.42e2e63f6bc5aa9128b8b31376576136.jpg
            DJI_0087_JPG_jpg.rf.368cba42a5bb2e26f2adde2e399d8ac1.jpg
            DJI_0274_JPG_jpg.rf.aad9bf6795e9f58b4ce6658edda0413c.jpg
            DJI_0027_JPG_jpg.rf.eabe2fee7be57c26e606e2b6da268982.jpg
            ...
        labels/
            DJI_0064_JPG_jpg.rf.710b0ae3d04ebc0ed1781e9d24f214f9.txt
            DJI_0078_JPG_jpg.rf.89f4c2a513f794dc96289c2fcad30cd7.txt
            DJI_0414_JPG_jpg.rf.2182f14a081b9adda3febafaad2a9b73.txt
            DJI_0536_JPG_jpg.rf.a2487e502fd

train: Scanning /content/test-17/train/labels.cache... 6282 images, 1515 backgrounds, 0 corrupt: 100%|██████████| 6282/6282 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.001), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     81/200      7.25G      2.573      3.976      1.655         41        640: 100%|██████████| 393/393 [01:57<00:00,  3.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:05<00:00,  3.47it/s]

                   all        607       1447      0.409      0.419      0.315      0.114



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     82/200      7.35G      2.547      3.887      1.641         88        640:  55%|█████▍    | 215/393 [01:04<00:54,  3.28it/s]

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pip install ultralytics
!pip install roboflow
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122

import os
import yaml
import json
import shutil
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
import matplotlib.pyplot as plt
from roboflow import Roboflow

# Initialize Roboflow with API key
rf = Roboflow(api_key="HA7uKn41BYAjKQOtGKJz")
project = rf.workspace("capstone-ncwsi").project("test-5dvot")
version = project.version("17")

dataset = version.download("yolov8")

# Now we can use dataset.location
yaml_path = os.path.join(dataset.location, "data.yaml")

def print_directory_structure(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files[:5]:  # Limiting to first 5 files to avoid clutter
            print('{}{}'.format(subindent, f))
        if len(files) > 5:
            print('{}...'.format(subindent))

print("Directory structure:")
print_directory_structure(dataset.location)

def fix_yaml_paths(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    base_dir = os.path.dirname(yaml_path)
    data['train'] = os.path.relpath(os.path.join(base_dir, 'train', 'images'), base_dir)
    data['val'] = os.path.relpath(os.path.join(base_dir, 'valid', 'images'), base_dir)
    data['test'] = os.path.relpath(os.path.join(base_dir, 'test', 'images'), base_dir)

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)

    print("YAML file updated.")

fix_yaml_paths(yaml_path)

print("\nYAML file content:")
with open(yaml_path, 'r') as f:
    print(f.read())

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

class CustomYOLO(YOLO):
    def __init__(self, model='yolov8m.pt', task=None):
        super().__init__(model, task)
        self.focal_loss = FocalLoss(alpha=0.25, gamma=2.0)

    def _load(self, weights='yolov8m.pt', task='detect'):
        super()._load(weights, task)

        if hasattr(self.model, 'model') and hasattr(self.model.model, 'loss'):
            original_loss_func = self.model.model.loss
            def custom_loss(*args, **kwargs):
                loss, loss_items = original_loss_func(*args, **kwargs)

                pred_scores, target_scores = args[0][1], args[1]['cls']
                focal_loss = self.focal_loss(pred_scores, target_scores)
                loss_items[1] = focal_loss
                loss = loss_items.sum()
                return loss, loss_items
            self.model.model.loss = custom_loss

def on_train_epoch_end(trainer):
    epoch = trainer.epoch
    if epoch % trainer.args.save_period == 0:
        best_fitness = trainer.best_fitness if trainer.best_fitness is not None else 0.0
        extra_info = {
            'epoch': epoch,
            'best_fitness': float(best_fitness),
            'learning_rate': float(trainer.optimizer.param_groups[0]['lr']),
        }
        save_dir = Path(trainer.args.project) / trainer.args.name
        info_path = save_dir / f'extra_info_epoch_{epoch}.json'
        latest_info_path = save_dir / 'extra_info_latest.json'
        with open(info_path, 'w') as f:
            json.dump(extra_info, f)
        with open(latest_info_path, 'w') as f:
            json.dump(extra_info, f)

        # Save a backup of the checkpoint
        checkpoint_path = save_dir / 'weights' / f'epoch{epoch}.pt'
        backup_path = save_dir / 'weights' / f'epoch{epoch}_backup.pt'
        if checkpoint_path.exists():
            shutil.copy(str(checkpoint_path), str(backup_path))

def find_latest_checkpoint(save_dir):
    weights_dir = save_dir / 'focal_loss_final' / 'weights'
    checkpoints = list(weights_dir.glob('epoch*.pt'))
    if not checkpoints:
        return None
    return max(checkpoints, key=lambda x: int(x.stem.replace('epoch', '')))

def train_and_evaluate_with_focal_loss(yaml_path, save_dir, alpha, gamma):
    weights_dir = save_dir / 'focal_loss_final' / 'weights'
    latest_checkpoint = find_latest_checkpoint(save_dir)

    if latest_checkpoint:
        print(f"Resuming training from {latest_checkpoint}")
        model = CustomYOLO(latest_checkpoint)
        start_epoch = int(latest_checkpoint.stem.replace('epoch', ''))
        total_epochs = 200  
        remaining_epochs = total_epochs - start_epoch
    else:
        print("Starting a new training session.")
        model = CustomYOLO('yolov8m.pt')
        remaining_epochs = 200

    model.focal_loss = FocalLoss(alpha=alpha, gamma=gamma)
    os.makedirs(str(weights_dir), exist_ok=True)
    model.add_callback('on_train_epoch_end', on_train_epoch_end)

    results = model.train(
        data=yaml_path,
        epochs=remaining_epochs,
        imgsz=640,
        batch=16,
        lr0=0.001,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.001,
        warmup_epochs=5,
        warmup_bias_lr=0.1,
        warmup_momentum=0.8,
        box=10.0,
        cls=1.0,
        dfl=1.5,
        patience=100,
        optimizer="AdamW",
        cos_lr=True,
        mixup=0.5,
        copy_paste=0.3,
        degrees=15.0,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        translate=0.2,
        scale=0.6,
        shear=3.0,
        perspective=0.0005,
        flipud=0.5,
        fliplr=0.5,
        mosaic=1.0,
        conf=0.001,
        iou=0.5,
        verbose=True,
        save_period=10,
        label_smoothing=0.1,
        project=str(save_dir),
        name='focal_loss_final',
        exist_ok=True,
        resume=True,
        device='0'  # Specify the GPU device if needed
    )

    val_results = model.val(data=yaml_path)
    return results, val_results

# Set the save directory
save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329')

# Set the best alpha and gamma values
best_alpha = 0.25
best_gamma = 1.0

# Run the training and evaluation
results, val_results = train_and_evaluate_with_focal_loss(yaml_path, save_dir, best_alpha, best_gamma)

# Print final validation results
if val_results:
    print(f"\nFinal validation results:")
    print(f"mAP50: {val_results.box.map50:.4f}")
    print(f"mAP50-95: {val_results.box.map:.4f}")
    print(f"Precision: {val_results.box.p:.4f}")
    print(f"Recall: {val_results.box.r:.4f}")
else:
    print("Validation results not available.")

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-


Extracting Dataset Version Zip to test-17 in yolov8:: 100%|██████████| 14386/14386 [00:02<00:00, 5327.57it/s]


Directory structure:
test-17/
    data.yaml
    README.dataset.txt
    README.roboflow.txt
    train/
        images/
            DJI_0413_JPG_jpg.rf.19320fe26fc336bc6e2cf73616110b57.jpg
            DJI_0146_JPG_jpg.rf.892170fa4020df67364d56e266ba52d6.jpg
            DJI_0306_JPG_jpg.rf.a78f10c66fbf4953f05ebc164efbf4b2.jpg
            DJI_0519_JPG_jpg.rf.15e5b50e7689833e3c461c2e94a67901.jpg
            DJI_0869_JPG_jpg.rf.18b23757c4f593029a04c5e4e29aedb9.jpg
            ...
        labels/
            DJI_0240_JPG_jpg.rf.3ed41262ebea27084d056a81761242b5.txt
            DJI_0108_JPG_jpg.rf.d841b2c98b46cf88998eb0e4c2e15ebe.txt
            DJI_0016_JPG_jpg.rf.f973ffad9dbd12989643018bba6fdf88.txt
            DJI_0902_JPG_jpg.rf.8d2162c5f23680dc48bd8423faa2ea52.txt
            DJI_0566_JPG_jpg.rf.88d2d5d7890302b1d231cf5e21e1cfe5.txt
            ...
    valid/
        images/
            DJI_0362_JPG_jpg.rf.0793488635815e15b3833228e6d09996.jpg
            DJI_0952_JPG_jpg.rf.19ec99611d80d7ac

100%|██████████| 755k/755k [00:00<00:00, 28.5MB/s]


TensorBoard: Start with 'tensorboard --logdir /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final', view at http://localhost:6006/

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytic

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  4.14it/s]


                   all        607       1447      0.409      0.361      0.345      0.127
      Beetle-fire Tree         63         79       0.45      0.405      0.428      0.165
             Dead Tree        297        550      0.471      0.481      0.424      0.159
                debris        265        818      0.305      0.197      0.184      0.057
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final
Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:05<00:00,  6.64it/s]


                   all        607       1447      0.408       0.36      0.345      0.128
      Beetle-fire Tree         63         79       0.45      0.405      0.427      0.166
             Dead Tree        297        550      0.471       0.48      0.424       0.16
                debris        265        818      0.304      0.196      0.185     0.0572
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final

Final validation results:
mAP50: 0.3452
mAP50-95: 0.1276


TypeError: unsupported format string passed to numpy.ndarray.__format__

In [4]:
import numpy as np

# Set the save directory
save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329')

# Set the best alpha and gamma values
best_alpha = 0.25
best_gamma = 1.0

# Run the training and evaluation
results, val_results = train_and_evaluate_with_focal_loss(yaml_path, save_dir, best_alpha, best_gamma)

# Print final validation results
if val_results:
    print(f"\nFinal validation results:")
    print(f"mAP50: {val_results.box.map50:.4f}")
    print(f"mAP50-95: {val_results.box.map:.4f}")

    # Handle precision
    if isinstance(val_results.box.p, np.ndarray):
        print(f"Precision: {val_results.box.p.mean():.4f}")
    else:
        print(f"Precision: {val_results.box.p:.4f}")

    # Handle recall
    if isinstance(val_results.box.r, np.ndarray):
        print(f"Recall: {val_results.box.r.mean():.4f}")
    else:
        print(f"Recall: {val_results.box.r:.4f}")

    # Print per-class results
    print("\nPer-class results:")
    for i, (p, r, map50, map) in enumerate(zip(val_results.box.p, val_results.box.r, val_results.box.map50, val_results.box.map)):
        class_name = val_results.names[i] if hasattr(val_results, 'names') else f"Class {i}"
        print(f"{class_name}:")
        print(f"  Precision: {p:.4f}")
        print(f"  Recall: {r:.4f}")
        print(f"  mAP50: {map50:.4f}")
        print(f"  mAP50-95: {map:.4f}")
else:
    print("Validation results not available.")

# Print additional information if available
if hasattr(val_results, 'speed'):
    print(f"\nInference speed: {val_results.speed['inference']:.2f} ms/image")

Resuming training from /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final/weights/epoch180.pt
Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final/weights/epoch180.pt, data=/content/test-17/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=0, workers=8, project=/content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329, name=focal_loss_final, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=/content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final/weights/epoch180.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json

train: Scanning /content/test-17/train/labels.cache... 6282 images, 1515 backgrounds, 0 corrupt: 100%|██████████| 6282/6282 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.001), 83 bias(decay=0.0)
Resuming training /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final/weights/epoch180.pt from epoch 182 to 200 total epochs
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  4.15it/s]


                   all        607       1447      0.409      0.361      0.345      0.127
      Beetle-fire Tree         63         79       0.45      0.405      0.428      0.165
             Dead Tree        297        550      0.471      0.481      0.424      0.159
                debris        265        818      0.305      0.197      0.184      0.057
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final
Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:05<00:00,  7.18it/s]


                   all        607       1447      0.408       0.36      0.345      0.128
      Beetle-fire Tree         63         79       0.45      0.405      0.427      0.166
             Dead Tree        297        550      0.471       0.48      0.424       0.16
                debris        265        818      0.304      0.196      0.185     0.0572
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final

Final validation results:
mAP50: 0.3452
mAP50-95: 0.1276
Precision: 0.4083
Recall: 0.3605

Per-class results:


TypeError: 'numpy.float64' object is not iterable

In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


!pip install ultralytics
!pip install roboflow
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122



import json
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from roboflow import Roboflow
!pip install ultralytics
!pip install roboflow
import os
import yaml
from ultralytics import YOLO
import numpy as np

# Initialize Roboflow with API key
rf = Roboflow(api_key="HA7uKn41BYAjKQOtGKJz")
project = rf.workspace("capstone-ncwsi").project("test-5dvot")
version = project.version("17")

dataset = version.download("yolov8")

yaml_path = "/content/test-17/data.yaml"

def fix_yaml_paths(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)


    base_dir = os.path.dirname(yaml_path)
    data['train'] = os.path.relpath(os.path.join(base_dir, 'train', 'images'), base_dir)
    data['val'] = os.path.relpath(os.path.join(base_dir, 'valid', 'images'), base_dir)
    data['test'] = os.path.relpath(os.path.join(base_dir, 'test', 'images'), base_dir)

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)

    print("YAML file updated.")


fix_yaml_paths(yaml_path)

print("\nYAML file content:")
with open(yaml_path, 'r') as f:
    print(f.read())
save_dir = '/content/drive/MyDrive/YOLOv8_nadir_focal_loss_0730_0329/focal_loss_final'
best_model_path = os.path.join(save_dir, 'weights', 'best.pt')

# Load the YOLO model
model = YOLO(best_model_path)

# Validate the model
val_results = model.val(data=yaml_path, verbose=True)

def calculate_mar(val_results):
    if not hasattr(val_results, 'box') or not hasattr(val_results.box, 'r'):
        print("Error: Validation results don't have the expected structure.")
        return

    recalls = val_results.box.r

    # compute mAR
    if isinstance(recalls, np.ndarray):
        mar = np.mean(recalls)
    else:
        mar = recalls

    print(f"\nmAR: {mar:.4f}")

    print("\nOther metrics:")
    print(f"mAP50: {val_results.box.map50:.4f}")
    print(f"mAP50-95: {val_results.box.map:.4f}")

    # Precision
    if isinstance(val_results.box.p, np.ndarray):
        print(f"Overall Precision: {val_results.box.p.mean():.4f}")
    else:
        print(f"Overall Precision: {val_results.box.p:.4f}")

    # Recall
    if isinstance(val_results.box.r, np.ndarray):
        print(f"Overall Recall: {val_results.box.r.mean():.4f}")
    else:
        print(f"Overall Recall: {val_results.box.r:.4f}")

    print("\nDetailed class-wise metrics:")
    if hasattr(val_results, 'names'):
        for i, name in enumerate(val_results.names):
            print(f"{name}:")
            if isinstance(val_results.box.p, np.ndarray):
                print(f"  Precision: {val_results.box.p[i]:.4f}")
            if isinstance(val_results.box.r, np.ndarray):
                print(f"  Recall: {val_results.box.r[i]:.4f}")
                print(f"  mAR: {val_results.box.r[i]:.4f}")  # mAR is the same as Recall for each class
            if isinstance(val_results.box.map50, np.ndarray):
                print(f"  mAP50: {val_results.box.map50[i]:.4f}")
            if isinstance(val_results.box.map, np.ndarray):
                print(f"  mAP50-95: {val_results.box.map[i]:.4f}")
    else:
        print("Class-wise metrics not available.")

calculate_mar(val_results)

Mounted at /content/drive
Looking in indexes: https://download.pytorch.org/whl/cu122
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.70, to fix: `pip install ultralytics==8.0.196`
YAML file updated.

YAML file content:
names:
- Beetle-fire Tree
- Dead Tree
- debris
nc: 3
roboflow:
  license: CC BY 4.0
  project: test-5dvot
  url: https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/17
  version: 17
  workspace: capstone-ncwsi
test: test/images
train: train/images
val: valid/images

Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 126MB/s]
val: Scanning /content/test-17/valid/labels... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<00:00, 1421.33it/s]

val: New cache created: /content/test-17/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:07<00:00,  5.27it/s]


                   all        607       1447      0.408      0.314      0.317      0.125
      Beetle-fire Tree         63         79      0.435      0.354      0.395      0.163
             Dead Tree        297        550      0.435      0.433      0.391      0.157
                debris        265        818      0.354      0.154      0.163     0.0549
Speed: 0.4ms preprocess, 2.9ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/val3

mAR: 0.3137

Other metrics:
mAP50: 0.3168
mAP50-95: 0.1250
Overall Precision: 0.4080
Overall Recall: 0.3137

Detailed class-wise metrics:
0:
  Precision: 0.4345
  Recall: 0.3544
  mAR: 0.3544
1:
  Precision: 0.4353
  Recall: 0.4327
  mAR: 0.4327
2:
  Precision: 0.3541
  Recall: 0.1540
  mAR: 0.1540
